In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime

In [17]:
#Load the arrest data
nypd = pd.read_csv('NYPD_Arrest_Data__Year_to_Date_.csv')

In [18]:
#Look at the five rows
nypd.head()

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude
0,206892169,12/31/2019,907.0,"IMPAIRED DRIVING,DRUG",347.0,INTOXICATED & IMPAIRED DRIVING,VTL11920U4,M,M,33,0,25-44,M,WHITE,1000916,245710,40.841085,-73.939769
1,206888084,12/31/2019,739.0,"FRAUD,UNCLASSIFIED-FELONY",112.0,THEFT-FRAUD,PL 1908301,F,Q,101,0,25-44,M,BLACK HISPANIC,1055868,156522,40.596012,-73.742116
2,206890433,12/31/2019,122.0,"HOMICIDE, NEGLIGENT, VEHICLE,",102.0,HOMICIDE-NEGLIGENT-VEHICLE,PL 1251201,F,M,23,0,18-24,M,BLACK,997462,227024,40.789803,-73.952289
3,206890441,12/31/2019,268.0,CRIMINAL MIS 2 & 3,121.0,CRIMINAL MISCHIEF & RELATED OF,PL 1450502,F,S,120,0,18-24,M,BLACK,962822,174282,40.645023,-74.077217
4,206890973,12/31/2019,101.0,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,M,25,97,25-44,M,BLACK,1004138,226326,40.787875,-73.928182


In [19]:
#The Data Dictionary for the Columns
data_dictionary = """
ARREST_KEY: Randomly generated persistent ID for each arrest

ARREST_DATE: Exact date of arrest for the reported event

PD_CD: Three digit internal classification code (more granular than
Key Code)

PD_DESC: Description of internal classification corresponding with PD
code (more granular than Offense Description)

KY_CD: Three digit internal classification code (more general
category than PD code)

OFNS_DESC: Description of internal classification corresponding with KY
code (more general category than PD description)

LAW_CODE: Law code charges corresponding to the NYS Penal Law,
VTL and other various local laws

LAW_CAT_CD: Level of offense: felony, misdemeanor, violation

ARREST_BORO: Borough of arrest. B(Bronx), S(Staten Island), K(Brooklyn),
M(Manhattan), Q(Queens)

ARREST_PRECINCT: Precinct where the arrest occurred

JURISDICTION_CODE: Jurisdiction responsible for arrest. Jurisdiction codes
0(Patrol), 1(Transit) and 2(Housing) represent NYPD whilst
codes 3 and more represent non NYPD jurisdictions

AGE_GROUP: Perpetrator’s age within a category

PERP_SEX: Perpetrator’s sex description

PERP_RACE: Perpetrator’s race description

X_COORD_CD: Midblock X-coordinate for New York State Plane
Coordinate System, Long Island Zone, NAD 83, units feet (FIPS 3104)

Y_COORD_CD: Midblock Y-coordinate for New York State Plane
Coordinate System, Long Island Zone, NAD 83, units feet (FIPS 3104)

Latitude: Latitude coordinate for Global Coordinate System, WGS
1984, decimal degrees (EPSG 4326)

Longitude: Longitude coordinate for Global Coordinate System, WGS
1984, decimal degrees (EPSG 4326)

"""


In [20]:
#Look at the shape
nypd.shape

(214617, 18)

In [21]:
#Look at the column names and types
nypd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214617 entries, 0 to 214616
Data columns (total 18 columns):
ARREST_KEY           214617 non-null int64
ARREST_DATE          214617 non-null object
PD_CD                214576 non-null float64
PD_DESC              214458 non-null object
KY_CD                214458 non-null float64
OFNS_DESC            214458 non-null object
LAW_CODE             214617 non-null object
LAW_CAT_CD           213248 non-null object
ARREST_BORO          214617 non-null object
ARREST_PRECINCT      214617 non-null int64
JURISDICTION_CODE    214617 non-null int64
AGE_GROUP            214617 non-null object
PERP_SEX             214617 non-null object
PERP_RACE            214617 non-null object
X_COORD_CD           214617 non-null int64
Y_COORD_CD           214617 non-null int64
Latitude             214617 non-null float64
Longitude            214617 non-null float64
dtypes: float64(4), int64(5), object(9)
memory usage: 29.5+ MB


For the most part there are no missing entries (almost 214,617 for all columns). It looks like half of the columns are numbers and half are strings (object)

In [22]:
#Drop the columns that are not needed for analysis
# nypd = nypd.drop(columns = [])

In [23]:
#Look at the data after dropping
# nypd.head()

In [24]:
#Look at the shape again
# nypd..shape

In [25]:
#Are there empty values?
nypd.isnull().values.any()

True

In [26]:
#How many empty values are there in each column?
nypd.isnull().sum()

ARREST_KEY              0
ARREST_DATE             0
PD_CD                  41
PD_DESC               159
KY_CD                 159
OFNS_DESC             159
LAW_CODE                0
LAW_CAT_CD           1369
ARREST_BORO             0
ARREST_PRECINCT         0
JURISDICTION_CODE       0
AGE_GROUP               0
PERP_SEX                0
PERP_RACE               0
X_COORD_CD              0
Y_COORD_CD              0
Latitude                0
Longitude               0
dtype: int64

In [27]:
#Add the above counts together
nypd.isnull().sum().sum()

1887

<font color ='purple'> After examining the NYPD dataset, we observed that there are 1887 missing values. </font>


In [28]:
#Because it is not many, we drop the null values
nypd = nypd.dropna()

<font color='purple'>We dropped all records with missing data, since filling it with zeros or the mean would not be the best choice, since no calculations will be used to analyse the data. Those numeric values are data that reflects real facts/numbers and should not be manipulated else it can result in misleading analysis. We only lost less than 1% of our data by dropping them. </font>


In [29]:
#After dropping the null values we look at the data info again
nypd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213089 entries, 0 to 214616
Data columns (total 18 columns):
ARREST_KEY           213089 non-null int64
ARREST_DATE          213089 non-null object
PD_CD                213089 non-null float64
PD_DESC              213089 non-null object
KY_CD                213089 non-null float64
OFNS_DESC            213089 non-null object
LAW_CODE             213089 non-null object
LAW_CAT_CD           213089 non-null object
ARREST_BORO          213089 non-null object
ARREST_PRECINCT      213089 non-null int64
JURISDICTION_CODE    213089 non-null int64
AGE_GROUP            213089 non-null object
PERP_SEX             213089 non-null object
PERP_RACE            213089 non-null object
X_COORD_CD           213089 non-null int64
Y_COORD_CD           213089 non-null int64
Latitude             213089 non-null float64
Longitude            213089 non-null float64
dtypes: float64(4), int64(5), object(9)
memory usage: 30.9+ MB


In [30]:
nypd['ARREST_DATE'] = pd.to_datetime(nypd['ARREST_DATE'], format = "%m/%d/%Y")


<font color = 'purple' > In 2019, the highest Offence was 'Assault related Offensed which made up 15%, followed by Petit Larceny with 10%.<\font>

In [31]:
#change borough columns to make it clear
borough= {'K': 'Brooklyn', 'M': 'Manhattan','B':'Bronx','Q':"Queens", 'S':'Staten Island'}
nypd['ARREST_BORO'] = nypd['ARREST_BORO'].replace(borough)

In [32]:
#Looking at the number of arrests by borough
arrest_by_boro = nypd.groupby('ARREST_BORO').OFNS_DESC.count()
arrest_by_boro = pd.DataFrame(arrest_by_boro.sort_values(ascending = False))

arrest_by_boro

,OFNS_DESC
ARREST_BORO,
Brooklyn,58077
Manhattan,53172
Bronx,48516
Queens,44329
Staten Island,8995


<font color = 'purple'> Brooklyn had 58,077 Offences, the highest number of Offences for the year 2019 as shown above, followed by Manhattan with 53,172, then the Bronx, Queens and Staten island, with 48,516, 44,329 and 8,995 respectively.<font/>

In [56]:
#Describe the numerical values
nypd.describe()

,ARREST_KEY,PD_CD,KY_CD,ARREST_PRECINCT,JURISDICTION_CODE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude
count,2.130890e+05,213089.000000,213089.000000,213089.000000,213089.000000,2.130890e+05,213089.000000,213089.000000,213089.000000
mean,1.989047e+08,452.744647,257.959332,62.548672,2.311813,1.005407e+06,208197.198889,40.738087,-73.923631
std,4.348545e+06,279.749363,141.631085,34.961465,13.578490,2.135185e+04,29941.634229,0.082188,0.077002
min,1.917037e+08,101.000000,101.000000,1.000000,0.000000,9.141030e+05,121152.000000,40.498957,-74.252251
25%,1.951565e+08,185.000000,117.000000,40.000000,0.000000,9.913600e+05,185685.000000,40.676286,-73.974367
50%,1.986640e+08,439.000000,341.000000,61.000000,0.000000,1.004905e+06,206894.000000,40.734485,-73.925436
75%,2.025553e+08,729.000000,344.000000,100.000000,0.000000,1.017440e+06,236071.000000,40.814612,-73.879998
max,2.068936e+08,997.000000,882.000000,123.000000,97.000000,1.067302e+06,271304.000000,40.911307,-73.700293


In [37]:
#How old are those arrested?
#First lets see how age is stored:
nypd.AGE_GROUP.unique()

array(['25-44', '18-24', '45-64', '<18', '65+'], dtype=object)

There are 5 categorical variables for age group of person

In [38]:
#Counting the persons in each age category
nypd.AGE_GROUP.value_counts()

25-44    113292
18-24     43275
45-64     43249
<18       10260
65+        3013
Name: AGE_GROUP, dtype: int64

Above it shows that most criminals are in the 25 to 44 age group

In [40]:
#Count the crimes by the different races
nypd.PERP_RACE

In [51]:
#Count the crimes by the different genders
nypd.PERP_SEX.value_counts()

M    173964
F     39125
Name: PERP_SEX, dtype: int64

In [48]:
#What is the relationship between level_of_offense (felony, misdemeanor) and race?

# nypd.LAW_CAT_CD
nypd.PERP_RACE.value_counts()

BLACK                             101979
WHITE HISPANIC                     53265
WHITE                              25284
BLACK HISPANIC                     18550
ASIAN / PACIFIC ISLANDER           12013
UNKNOWN                             1401
AMERICAN INDIAN/ALASKAN NATIVE       597
Name: PERP_RACE, dtype: int64

In [43]:
#How about level_of_offense and borough?
# nypd.ARREST_BORO

In [47]:
#How about level_of_offense and age?
# nypd.AGE_GROUP